In [16]:
class Node:
    def __init__(self,data,level,fval):
        """ Initialize the node with the data, level of the node and the calculated fvalue """
        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self, prev_move_direction: str):
        # Generate child nodes from the given node by moving the blank space either in the four directions {left,up,right,down}
        # val_list contains position values for moving the blank space in either of the 4 directions [left,up,right,down] respectively.
        x, y = self.find(self.data, '_')
        direction_map = {
            "1_left": [x, y - 1],
            "2_up": [x - 1, y],
            "3_right": [x, y + 1],
            "4_down": [x + 1, y]
        }
        opposite_direction_map = {
            "1_left": "3_right",
            "2_up": "4_down",
            "3_right": "1_left",
            "4_down": "2_up",
        }
        child_list = []
        for k,v in direction_map.items():
            if opposite_direction_map[k] != prev_move_direction:
                child = self.shuffle(self.data, x, y, v[0], v[1])
                if child is not None:
                    child_node = Node(child,self.level+1,0)
                    child_list.append(
                        {
                            "node": child_node,
                            "f": _get_f(child_node),
                            "prev_move_direction": k
                         }
                    )

        return child_list
        
    def shuffle(self,puz,x1,y1,x2,y2):
        """ Move the blank space in the given direction and if the position value are out
            of limits the return None """
        if x2 >= 0 and x2 < len(self.data) and y2 >= 0 and y2 < len(self.data):
            temp_puz = []
            temp_puz = self.copy(puz)
            temp = temp_puz[x2][y2]
            temp_puz[x2][y2] = temp_puz[x1][y1]
            temp_puz[x1][y1] = temp
            return temp_puz
        else:
            return None
            

    def copy(self,root):
        """ Copy function to create a similar matrix of the given node"""
        temp = []
        for i in root:
            t = []
            for j in i:
                t.append(j)
            temp.append(t)
        return temp    
            
    def find(self,puz,x):
        """ Specifically used to find the position of the blank space """
        for i in range(0,len(self.data)):
            for j in range(0,len(self.data)):
                if puz[i][j] == x:
                    return i,j

def _get_f(node: Node):
    return _get_h(node) + node.level

def _get_h(node: Node, type_: str = "h1"):
    """ Calculates the different between the given puzzles """
    goal = [["_", 1, 2], [3, 4, 5], [6, 7, 8]]
    temp = 0
    if type_ == "h1":
        for i in range(0,3):
            for j in range(0,3):
                if node.data[i][j] != goal[i][j] and node.data[i][j] != '_':
                    temp += 1
    elif type_ == "h2":
        pass
    return temp

class Puzzle:
    def __init__(self,size):
        self.n = size

    def f(self,start,goal):
        """ Heuristic Function to calculate hueristic value f(x) = h(x) + g(x) """
        return self.h(start.data,goal)+start.level

    def h(self,start,goal):
        """ Calculates the different between the given puzzles """
        temp = 0
        for i in range(0,self.n):
            for j in range(0,self.n):
                if start[i][j] != goal[i][j] and start[i][j] != '_':
                    temp += 1
        return temp
        

    def process(self):
        # initialization
        start = [[7, 2, 4], [5, "_", 6], [8, 3, 1]]
        goal = [["_", 1, 2], [3, 4, 5], [6, 7, 8]]
        start = Node(start,0,0)
        start.fval = self.f(start,goal)
        node_queue = [{"node": start, "f": start.fval, "prev_move_direction": ""}]

        iter = 0
        while iter < 3:
            iter += 1

            current_node = node_queue.pop(0)
            print("\n \n ------------------------------------------ \n \n")
            print(f"iteration {iter}")
            for i in current_node["node"].data:
                for j in i:
                    print(j,end=" ")
                print("")

            # Check if the current node is already the same with the goal
            if(self.h(current_node["node"].data, goal) == 0):
                break

            print(node_queue)
            print(current_node["node"].generate_child(current_node["prev_move_direction"]))
            # adding new child to infront of the queue
            node_queue = current_node["node"].generate_child(current_node["prev_move_direction"]) + node_queue
            # sort the queue based on f value
            node_queue.sort(key=lambda x: x["f"])
            print(node_queue)



puz = Puzzle(3)
puz.process()


 
 ------------------------------------------ 
 

iteration 1
7 2 4 
5 _ 6 
8 3 1 
[]
[{'node': <__main__.Node object at 0x7fb8dc35dcd0>, 'f': 9, 'prev_move_direction': '1_left'}, {'node': <__main__.Node object at 0x7fb8ec70ab50>, 'f': 9, 'prev_move_direction': '2_up'}, {'node': <__main__.Node object at 0x7fb8ec70acd0>, 'f': 9, 'prev_move_direction': '3_right'}, {'node': <__main__.Node object at 0x7fb8ee73e850>, 'f': 9, 'prev_move_direction': '4_down'}]
[{'node': <__main__.Node object at 0x7fb8dc320e80>, 'f': 9, 'prev_move_direction': '1_left'}, {'node': <__main__.Node object at 0x7fb8ee73e850>, 'f': 9, 'prev_move_direction': '2_up'}, {'node': <__main__.Node object at 0x7fb8ec70acd0>, 'f': 9, 'prev_move_direction': '3_right'}, {'node': <__main__.Node object at 0x7fb8ec70a6a0>, 'f': 9, 'prev_move_direction': '4_down'}]

 
 ------------------------------------------ 
 

iteration 2
7 2 4 
_ 5 6 
8 3 1 
[{'node': <__main__.Node object at 0x7fb8ee73e850>, 'f': 9, 'prev_move_direction': '2